In [1]:
!pip install underthesea

# old code:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector, Softmax, Lambda, TimeDistributed, Layer, MultiHeadAttention, Reshape, Input
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np

  Obtaining dependency information for underthesea from https://files.pythonhosted.org/packages/c0/26/a2615fb23899ec747eaf7d7b3d4ae6b1959b71ae2a8546e121a8ce375d9c/underthesea-6.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for python-crfsuite>=0.9.6 from https://files.pythonhosted.org/packages/38/1d/c475ba7d11e9735f00eb08e2f5315aa2e21c24cc85a0474c3fd425edef58/python_crfsuite-0.9.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for underthesea-core==1.0.4 from https://files.pythonhosted.org/packages/3c/7e/b5af0cc8c567e0fc545a897850d8c216df2c2d02f30a52cbe60da252ce92/underthesea_core-1.0.4-cp310-cp310-manylinux2010_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.8 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#new code:
import tensorflow as tf
import pandas as pd
import numpy as np
import re,string
from gensim.models import KeyedVectors
from collections import Counter
from underthesea import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.losses import categorical_crossentropy
from keras import utils
from tensorflow.keras import preprocessing, utils, activations
from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow_datasets as tfds

In [4]:
from underthesea import word_tokenize

In [5]:
k = word_tokenize('Ngày hôm qua, tôi đã xem phim trinh thám! Tôi rất buồn')

In [6]:
print(k)

['Ngày', 'hôm qua', ',', 'tôi', 'đã', 'xem', 'phim', 'trinh thám', '!', 'Tôi', 'rất', 'buồn']


In [7]:
def check_tieng_viet(seq):
    intab_l = 'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđ'
    ascii_lowercase = 'abcdefghijklmnopqrstuvwxyz'
    digits = '0123456789'
    punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
    whitespace = ' '
    accept_strings = intab_l + ascii_lowercase + digits + punctuation + whitespace
    r = re.compile('^[' + accept_strings + ']+$')

    if re.match(r, seq.lower()):
        return True
    return False

In [8]:
from tqdm import tqdm
def remove_tone_line(utf8_str):
    intab_l = 'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđ'
    intab_u = 'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸĐ'
    intab = list(intab_l + intab_u)
    outtab_l = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"
    outtab_u = "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"
    outtab = outtab_l + outtab_u
    r = re.compile("|".join(intab))
    replaces_dict = dict(zip(intab, outtab))
    non_dia_str = r.sub(lambda m: replaces_dict[m.group(0)], utf8_str)
    return non_dia_str


In [9]:
def read_data(file_path):
    train_output = None
    with open(file_path, 'r', encoding='utf8') as f:
        train_output = f.readlines()
    return train_output

In [10]:
# Không chạy:
from underthesea import word_tokenize
def create_word2index(output_data):
    word2index = {}
    index2word = {}
    index2word[0] = '</pad>'
    index2word[1] = '</Start>'
    index2word[2] = '</EOS>'
    index2word[3] = '</unkn>'
    word2index['</pad>'] = 0
    word2index['</Start>'] = 1
    word2index['</EOS>'] = 2
    word2index['</unkn>'] = 3
    index = 4
    print('index2word tokenizing!!!')
    for idx_of_file in tqdm(range(len(output_data))):
        # seq = output_data[idx_of_file]
        # text = seq.split('\t')[0]
        text = output_data[idx_of_file]
        tokenized_word = word_tokenize(text)
        for item in tokenized_word:
            if(item not in index2word.values()):
                index2word[index] = item
                word2index[item] = index
                index+=1
    return (word2index, index2word)


In [11]:
output_data = read_data(r'/kaggle/input/train-tieng-viet-v2-txt/train_tieng_viet_v2.txt')
# read_data(r'/content/drive/MyDrive/WorkSpace/data/train_data/train_tieng_viet.txt')

print(len(output_data))
print(output_data[0])

328034
0000000000	thế vận hội mùa đông 2010



In [12]:
import glob
import json
def create_output_data(folder_path):
    index = 0
    for path in sorted(glob.glob(f'{folder_path}/*')):
        for path1 in tqdm(sorted(glob.glob(f'{path}/*'))):
            with open(path1, 'r', encoding='utf8') as f:
                content = f.readlines()
                for row in content:
                    art_json = json.loads(row)
                    art_cont = art_json['text']
                    # print('art bef: ', art_cont)
                    # trong định nghĩa json, \1 là 1 khoảng trắng. Chuẩn hóa nhiều khoảng trắng thành duy nhất 1.
                    art_cont = re.sub("(\s)+", r"\1", art_cont)
                    # print('art aft: ', art_cont)
                    # input('Tiep tuc?  ')
                    art_seqs = art_cont.split("\n")
                    for seq in art_seqs:
                        if check_tieng_viet(seq) and seq != '':
                            seq = re.sub(r'[^\w\s]', '', seq)

                            # Chuyển đổi văn bản thành chữ thường
                            seq = seq.lower()

                            index_str = str(index).zfill(10)
                            with open(r'/content/drive/MyDrive/WorkSpace/data/train_data/train_tieng_viet.txt', 'a', encoding= 'utf8') as f1:
                                f1.writelines([index_str+'\t', seq+'\n'])
                            index += 1

In [13]:
# create_output_data(r'/content/drive/MyDrive/WorkSpace/data/raw_data')
# Dữ liệu đã được tạo

In [14]:
data = []
maxlength = 30

for item in output_data:
   _, text = item.split('\t')
   if(len(text) <= maxlength):
    data.append(text)
print(len(data))
# input('hello')

74363


Chuẩn hóa dữ liệu:


In [15]:
#old:
len_data = 5000
data_v1 = data[:len_data]

print(data_v1[:10])

# word2index, index2word = create_word2index(data_v1)

['thế vận hội mùa đông 2010\n', 'thế vận hội mùa hè 2008\n', 'apollo\n', 'sân bay phù cát\n', 'apollo định hướng\n', 'apollo có nhiều nghĩa\n', 'tên apollo cũng được dùng cho\n', 'phật giáo hệ phả\n', 'mười đại đệ tử\n', 'thiền phái tìniđalưuchi\n']


In [16]:
#New:
len_data = 5000
data_v1 = data[:len_data]

print(data_v1[:10])


['thế vận hội mùa đông 2010\n', 'thế vận hội mùa hè 2008\n', 'apollo\n', 'sân bay phù cát\n', 'apollo định hướng\n', 'apollo có nhiều nghĩa\n', 'tên apollo cũng được dùng cho\n', 'phật giáo hệ phả\n', 'mười đại đệ tử\n', 'thiền phái tìniđalưuchi\n']


In [1]:
#old:
# print(len(word2index))

In [ ]:
#new:
outputs = [['</start>'] + word_tokenize(output) + ['</end>'] for output in data_v1]
outputs.append(['</start>','</unknown>', '</end>'])
print(len(outputs))
print(outputs[-2:])

In [ ]:
# inputs = []

# for item in outputs:
#     input_item = []
#     for word in item:
#         input_item.append(remove_tone_line(word))
# #     input_item.append('</end>')
#     inputs.append(input_item)
# print(len(inputs))
# print(inputs[:10])
# # input('hello')

inputs = [['</start>'] + word_tokenize(remove_tone_line(output)) + ['</end>'] for output in data_v1]
inputs.append(['</start>','</unknown>', '</end>'])
print(len(inputs))
print(inputs[:10])


In [ ]:
#New:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(inputs + outputs)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

In [ ]:
#new
word2idx = tokenizer.word_index
# print(word2idx)

In [ ]:
#encoder_input_data
tokenized_inputs = tokenizer.texts_to_sequences(inputs)
maxlen_inputs = max([len(x) for x in tokenized_inputs])
padded_inputs = pad_sequences(tokenized_inputs, maxlen = maxlen_inputs, padding = 'post')
encoder_input_data = np.array(padded_inputs)
print("Max length input:", maxlen_inputs)
print(encoder_input_data.shape)

In [ ]:
# new
# decoder_input_data
tokenized_outputs = tokenizer.texts_to_sequences(outputs)
maxlen_outputs = max([len(x) for x in tokenized_outputs])
padded_outputs = pad_sequences(tokenized_outputs, maxlen = maxlen_outputs, padding='post')
decoder_input_data = np.array(padded_outputs)
print("Max length outputs:", maxlen_outputs)
print(decoder_input_data.shape)

In [ ]:
# new:
# decoder_output_data
tokenized_outputs = tokenizer.texts_to_sequences(outputs)
# Remove Start added before
for i in range(len(tokenized_outputs)):
    tokenized_outputs[i] = tokenized_outputs[i][1:]
padded_outputs = pad_sequences(tokenized_outputs, maxlen = maxlen_outputs, padding='post')
onehot_outputs = utils.to_categorical(padded_outputs, VOCAB_SIZE)
decoder_output_data = np.array(onehot_outputs)
print(decoder_output_data.shape)

In [ ]:
fastText_model = KeyedVectors.load_word2vec_format(r'/kaggle/input/fastext-vector/wiki.vi.vec')
print("FastText Loaded!")

In [ ]:
embeddings_dim = 300

embedding_matrix = np.zeros((VOCAB_SIZE, embeddings_dim))

for word, index in word2idx.items():
    try:
        embedding_matrix[index,:] = fastText_model[word]
    except:
        continue
        
print(embedding_matrix.shape)

In [ ]:
# print(maxlenght)

In [ ]:
# Define Embedding Layer
embedding_layer_inputs = Embedding(VOCAB_SIZE,embeddings_dim
                                     ,input_length=maxlen_inputs
                                     ,weights = [embedding_matrix]
                                     ,trainable=False)

embedding_layer_outputs = Embedding(VOCAB_SIZE,embeddings_dim
                                     ,input_length=maxlen_outputs
                                     ,weights = [embedding_matrix]
                                     ,trainable=False)

In [ ]:
# Define the model
encoder_inputs = Input(shape = (maxlen_inputs, ))
encoder_embedding = embedding_layer_inputs(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(300,dropout=0.05,return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(maxlen_inputs, ))
decoder_embedding = embedding_layer_outputs(decoder_inputs)
decoder_lstm = LSTM(300, return_state=True, return_sequences=True,dropout=0.05)


decoder_outputs , _ , _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)



decoder_dense = Dense(VOCAB_SIZE, activation='softmax') 
output = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='loss', verbose=1, save_best_only=True)

# train model
history = model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=64, epochs=100) 

In [ ]:
import joblib

def save_features(data, filepath):
    print('saving feature!!!')  
    joblib.dump(data, filepath)
    print('saved')

def load_features(file_path):
    print('loading file')
    print('loaded')
    return joblib.load(file_path)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 5)
# Plot training accuracy & loss values
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.title('Model accuracy')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(['Accuracy','Loss'], loc='upper left')
plt.show()

In [ ]:
model.save_weights('chatbot_weights.h5')

In [ ]:
def make_inference_models():
    
    encoder_model = Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(300,))
    decoder_state_input_c = Input(shape=(300,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model

In [ ]:
def str_to_tokens(sentence):
    words = word_tokenize(sentence.lower())
    tokens_list = []
    
    for word in words:
        try: 
            tokens_list.append(tokenizer.word_index[word])
        except Exception as e:
            tokens_list.append(tokenizer.word_index['</unknown>'])
    return pad_sequences([tokens_list],maxlen = maxlen_inputs , padding='post')

In [ ]:
enc_model ,dec_model = make_inference_models()

In [ ]:
save_features(enc_model, 'encoder_model.joblib')

save_features(dec_model, 'decoder_model.joblib')

In [ ]:
enc_model ,dec_model = make_inference_models()

flag=True

while(flag==True):
    human_response = input('Nhập câu không có dấu : ')
    if human_response != '':
#         print(human_response)
#         try:
        states_values = enc_model.predict(str_to_tokens(human_response))
        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = word2idx['</start>']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition:
            dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)

            sampled_word_index = np.argmax(dec_outputs[0, -1, :])
            sampled_word = None
            for word, index in word2idx.items() :
                if sampled_word_index == index:
                    decoded_translation += ' {}'.format(word)
                    sampled_word = word

            if sampled_word == '</end>' or len(decoded_translation.split()) > maxlen_outputs:
                stop_condition = True

            empty_target_seq = np.zeros((1, 1))  
            empty_target_seq[0 , 0] = sampled_word_index
            states_values = [h, c] 

        print('Res : ' + decoded_translation.replace("</end>",""))
    
    

In [ ]:
# old
print(data_v1[:10])


In [ ]:
# new:
print(tokenized_inputs[:10])

In [ ]:
target_data = []
for item in tqdm(data_v1):
  words = item.split('\n')[0]
  words = word_tokenize(words)
  vector =[]
  for word in words:
    if(word in word2index.keys()):
      vector.append(word2index[word])
    else:
      vector.append(word2index['</unkn>'])
  padding = maxlength - len(words)
  vector = vector + [0] * padding
  target_data.append(vector)

In [ ]:
sentences =[]
for item in target_data[:10]:
  words = []
  for index in item:
    words.append(index2word[index])
  sentence = ' '.join(words)
  sentences.append(sentence)
# for item in sentences:
#   print(item)



In [ ]:
import numpy as np
target_vector = np.array(target_data)
print(target_vector.shape)

In [ ]:
print(target_vector)

In [ ]:
def create_word2index_nonsys(word2index_sys):
  word2index = {}
  index2word = {}
  index2word[0] = '</pad>>'
  index2word[1] = '</Start>'
  index2word[2] = '</EOS>'
  index2word[3] = '</unkn>'
  word2index['</pad>'] = 0
  word2index['</Start>'] = 1
  word2index['</EOS>'] = 2
  word2index['</unkn>'] = 3
  index = 4
  for item in tqdm(word2index_sys.keys()):
    non_sys = remove_tone_line(item)
    if(non_sys not in word2index):
      word2index[non_sys] = index
      index2word[index] = non_sys
      index+= 1
  return (word2index, index2word)

In [ ]:
non_sys_word2index, non_sys_index2word = create_word2index_nonsys(word2index)
print(len(non_sys_word2index))

# print(non_sys_word2index.keys())

In [ ]:
input_data = []
for item in tqdm(data_v1):
  words = item.split('\n')[0]
  words = word_tokenize(words)
  vector =[]
  for word in words:
    word = remove_tone_line(word)
    if(word in non_sys_word2index.keys()):
      vector.append(non_sys_word2index[word])
    else:
      vector.append(non_sys_word2index['</unkn>'])
  padding = 30 - len(words)
  vector = vector + [0] * padding
  input_data.append(vector)

In [ ]:
sentences =[]
for item in input_data[:10]:
  words = []
  for index in item:
    words.append(non_sys_index2word[index])
  sentence = ' '.join(words)
  sentences.append(sentence)
# for item in sentences:
#   print(item)

In [ ]:
input_vector = np.array(input_data)
print(input_vector.shape)

In [ ]:
VOCAB_SIZE_input = len(non_sys_index2word)
VOCAB_SIZE_output = len(index2word)

embedding_dim = 300

In [ ]:
embedding_layer_input = Embedding(VOCAB_SIZE_input, embedding_dim,
                                     input_length=maxlength,
                                     trainable=False)

# Define Embedding Layer for Answers
embedding_layer_output = Embedding(VOCAB_SIZE_output, embedding_dim,
                                   input_length=maxlength,
                                   trainable=False)

In [ ]:
from keras.models import Model, Sequential

In [ ]:
print(len(word2index))

In [ ]:
encoder_inputs = Input(shape = (maxlength, ))
encoder_embedding = embedding_layer_input(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(300,dropout=0.05,return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(maxlength, ))
decoder_embedding = embedding_layer_output(decoder_inputs)
decoder_lstm = LSTM(300, return_state=True, return_sequences=True,dropout=0.05)
decoder_outputs , _ , _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE_output, activation='softmax')
output = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
def create_onehost_vector(target_vector, maxlength, word2index):
  onehost_vectors = np.zeros((len(target_vector),maxlength, len(word2index)))

  for n_vector in tqdm(range((len(target_vector)))):
    for index in range(maxlength):
      onehost_vectors[n_vector][index][target_vector[n_vector][index]] = 1
  print('shape: ', onehost_vectors.shape)
  return onehost_vectors

output_onehost_vector = create_onehost_vector(target_vector, maxlength, word2index)


In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='loss', verbose=1, save_best_only=True)

# train model
history = model.fit([input_vector , target_vector], output_onehost_vector , batch_size=64, epochs=200)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 5)
# Plot training accuracy & loss values
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.title('Model accuracy')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(['Accuracy','Loss'], loc='upper left')
plt.show()

In [ ]:
model.save_weights('chatbot_weights.h5')

In [ ]:
import pickle

In [ ]:
with open('/kaggle/working/data.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
print(input_vector[:2])

In [ ]:
def make_inference_models():
    
    encoder_model = Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(300,))
    decoder_state_input_c = Input(shape=(300,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model

In [ ]:
enc_model ,dec_model = make_inference_models()

In [ ]:
print(input_vector[:1])

In [ ]:
states_values = enc_model.predict(input_vector[104:105])
empty_target_seq = np.zeros((1, 1))
empty_target_seq[0, 0] = word2index['</Start>']
stop_condition = False
decoded_translation = ''
while not stop_condition:
    dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)

    sampled_word_index = np.argmax(dec_outputs[0, -1, :])
    sampled_word = None
    for word, index in word2index.items() :
        if sampled_word_index == index:
            decoded_translation += ' {}'.format(word)
            sampled_word = word

        if sampled_word == '</End>' or len(decoded_translation.split()) > 30:
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h, c]

In [ ]:
print(decoded_translation)
print(input)

In [ ]:
predicted_output = model.predict(input_vector[:2])

# # Kết quả là một vector dự đoán
print("Predicted Output Vector:", predicted_output)
print(data_v1[:2])

In [ ]:
def make_inference_models():
    
    encoder_model = Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(300,))
    decoder_state_input_c = Input(shape=(300,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model